# Training ML algos on EPH. Predicting on CENSO.

In [1]:
startyr = 2021
endyr = 2022

In [2]:
# Load modules
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from IPython.core.display import display, HTML

# import pickle
import joblib

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import json

In [3]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']
print(sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:5])


[('HTML', 1064), ('RandomForestClassifier', 1064), ('RandomForestRegressor', 1064), ('display', 136), ('train_test_split', 136)]


In [4]:
# Column names
y_cols = ['CAT_OCUP', 'P47T', 'PP10E', 'PP10D', 'PP07K', 'PP07I', 'V3_M', 'PP07G4', 'CH16', 'T_VI', 
          'V12_M', 'TOT_P12', 'PP07G3', 'V5_M', 'PP07H', 'V2_M', 'PP10C', 
          'PP08D1', 'PP07J', 'CAT_INAC', 'CH07', 'CH08', 'P21', 'PP07G1', 'PP07G_59', 'PP07G2']

x_cols = ['IX_TOT', 'P02', 'P03', 'AGLO_rk', 'Reg_rk', 'V01', 'H05', 'H06',
       'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14',
       'H13', 'P07', 'P08', 'P09', 'P10', 'P05', 'CONDACT']

x_cols1 = x_cols
predecir1 = ['CAT_OCUP', 'CAT_INAC', 'CH07']

x_cols2 = x_cols1 + predecir1
predecir2 = ['INGRESO', 'INGRESO_NLB', 'INGRESO_JUB', 'INGRESO_SBS']

x_cols3 = x_cols2 + predecir2
# La seccion PP07G pregunta si el trabajo es en blanco y que beneficios tiene. Puede ayudar a la regresion para ingresos.
# predecir3 = ['PP07G1', 'PP07G2', 'PP07G3', 'PP07G4', 'PP07G_59', 'PP07H', 'PP07I', 'PP07J', 'PP07K']
predecir3 = ['PP07G1','PP07G_59', 'PP07I', 'PP07J', 'PP07K']

# Columnas de ingresos. Necesitan una regresion...
columnas_pesos = [u'P21', u'P47T', u'PP08D1', u'TOT_P12', u'T_VI', u'V12_M', u'V2_M', u'V3_M', u'V5_M']
# P21: INGRESO DE LA OCUPACIÓN PRINCIPAL 
# P47T: TOTAL (laboral y no laboral)
# PP08D1: Sueldos, jornales, etc.
# TOT_P12: TOTAL DE INGRESO  POR  OTRAS OCUPACIONES (Secundaria, previa...)
# T_VI: TOTAL DE INGRESOS NO LABORALES
# V12_M:  CUOTAS DE ALIMENTOS O AYUDA EN DINERO  DE PERSONAS QUE NO VIVEN EN EL HOGAR 
# V2_M: ingreso por JUBILACION O PENSION
# V3_M: INDEMNIZACION POR DESPIDO 
# V5_M: SUBSIDIO  O AYUDA SOCIAL DEL GOBIERNO, IGLESIAS, ETC.

x_cols4 = x_cols3 + predecir3
# Columnas de ingresos. Necesitan una regresion...
predecir4 = columnas_pesos
y_cols4 = predecir4

In [5]:
def prepend_index_level(index, key, name=None):
    names = index.names
    if index.nlevels==1:
        # Sequence of tuples
        index = ((item,) for item in index)

    tuples_gen = ((key,)+item for item in index)
    return pd.MultiIndex.from_tuples(tuples_gen, names=[name]+names)

### Load info

In [6]:
radio_ref = pd.read_csv('./../data/info/radio_ref.csv')

AGLO_rk = pd.read_csv('./../data/info/AGLO_rk')
Reg_rk = pd.read_csv('./../data/info/Reg_rk')

### Set up Google sheet connection

In [7]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from numpy import array, nan

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
         './../../../Desktop/newgsheets-349817-e11b8c6a66ec.json', scope) # Your json file here
gc = gspread.authorize(credentials)
sheet_id = '1qlSesmbEnR0NHi_pe8NDJEG80v-fbumJBviy8yb9Ojc'
sh = gc.open_by_key(sheet_id)#.get_worksheet('Sheet1')

# # Get a list of all worksheets
# worksheet_list = sh.worksheets()


### Funcion subir confusion matrices a Gsheets

In [8]:
from sklearn.metrics import confusion_matrix  

def compute_upload_conf_matrix(y_true, y_pred, col):
    conf_vals = confusion_matrix(y_[col], y_pred[col])
    
    confusion = pd.DataFrame(conf_vals, columns = sorted(y_[col].unique()), index = sorted(y_pred[col].unique()))
    confusion = 100*confusion.div(confusion.sum(1), 0).round(3)
    
    confusion.columns.name = "Prediccion"
    confusion.index.name = "Verdadero"
    
    display(confusion)
    
    ## UPDATE G SHEET
    df = confusion.reset_index()
    rows, cols = df.shape

    worksheet = sh.worksheet(col)
    worksheet.update([df.columns.values.tolist()] + [6*['']] + df.values.tolist())
    worksheet.update('A1', 'Verdadero')
    worksheet.update('A2', 'Prediccion')
    worksheet.format('A', {'textFormat': {'bold': True}})
    worksheet.format('1', {'textFormat': {'bold': True}})

### Funcion entrenar modelo y guardarlo

In [9]:
def fit_save_clf(data, x_cols, y_cols, filename):

    data['split'] = data.index.values % 6

    test  = data.loc[data.split == 0].reset_index(drop = True)
    train = data.loc[data.split != 0]
    del data
    
    X, y = train[x_cols], train[y_cols]
    X_, y_ = test[x_cols], test[y_cols]
    del train; del test

    forest = RandomForestClassifier(n_estimators=100, max_depth = 20, n_jobs = -1)
    clf = forest.fit(X.values, y.values)

    # save the model to disk
    if not os.path.exists('./../fitted_RF/'):
        os.makedirs('./../fitted_RF/')
    
    joblib.dump(forest, filename, compress=3)
    
    del X; del y; # liberar memoria eliminando los dataframes mas pesados
    
    return clf, X_, y_  #devuelve modelo e info para testeo

### Funcion computar factores

In [10]:
def compute_factors_wCV(data, x_cols, y_cols):
    data['split'] = data.index.values % 6

    test  = data.loc[data.split == 0].reset_index(drop = True)
    train = data.loc[data.split != 0]
    
    ## En cada particion de Cross Validation
    factors_df_parts = []

    for i in range(1, 6)[:2]:
        print(i)
    #     Separar train y test
        cv_part = train.loc[train.split != i]
        cv_test = train.loc[train.split == i].reset_index(drop = True)

        # Ajustar modelo en train
        X, y = cv_part[x_cols], cv_part[y_cols]
        del cv_part

        forest = RandomForestClassifier(n_estimators=100, max_depth = 20, n_jobs = -1)
        clf = forest.fit(X.values, y.values)

        X_, y_ = cv_test[x_cols], cv_test[y_cols]
        del cv_test

        # # For each of the columns, get the proba so that the correct number of observations are above.
        proba_values = clf.predict_proba(X_)
    #     y_pred = y_.copy()

        for j, y_col in enumerate(y_cols):
            y_probas = pd.DataFrame(proba_values[j], columns = sorted(y_[y_col].unique()))

            counts = y_[y_col].value_counts().sort_index()

            ## Get the highest
            y_probas_rk = y_probas.rank(method = 'first', ascending = False)

            factors = ((y_probas_rk > counts)*y_probas).max()
            factors_df_part = pd.DataFrame(factors.reset_index()); 
            factors_df_part.columns = ['valor', 'factor']
            factors_df_part['variable'] = y_col
            factors_df_part['part'] = i
    #         y_pred[y_col] = (y_probas/factors).idxmax(1)#.value_counts().sort_index()

            factors_df_parts += [factors_df_part]

    factors_df = pd.concat(factors_df_parts)
    
    factors_mean = factors_df.groupby(['variable', 'valor'])['factor'].mean()
    
    return factors_mean

# Training

## Classification 1
Columnas ['CAT_OCUP', 'CAT_INAC', 'CH07']

In [11]:
# Load Census
### STEP 1
x_cols1 = x_cols
y_cols1 = predecir1


### Para cada anio. 
- Calcular Factores en Cross Validation y guardar.

In [ ]:
# if not exists, create /factors/

In [13]:
for yr in [str(s) for s in range(startyr, endyr)]:
    # Load EPH
    print(yr)
    data = pd.read_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv')
    print(data.shape)

    # Computar factores
    factors_mean = compute_factors_wCV(data, x_cols = x_cols1, y_cols = y_cols1)
    del data

    ## Guardar factores en archivo json
    tag = 'clf1_'+yr+'_ARG'
    factors_dict = dict()
    factors_dict[tag] = factors_mean.unstack().to_json()

    with open('./../data/training/factors/'+tag+'.json', 'w') as file: 
        json.dump(factors_dict, file)


2021
(357978, 56)
1
2


In [ ]:
# pd.DataFrame(proba_values[j]).sum()

In [14]:
# sorted(y_[y_col].unique())

In [15]:
# pd.DataFrame(proba_values[j], columns = sorted(y_[y_col].unique()))

In [16]:
print(sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:5])

[('radio_ref', 16764983), ('AGLO_rk', 14976), ('Reg_rk', 9948), ('RandomForestClassifier', 2008), ('ServiceAccountCredentials', 2008)]


### Para cada anio. 
- Ajustar modelo y guardar

In [17]:
# Entrenar modelo en train
for yr in [str(s) for s in range(startyr, endyr)]:
    # Load EPH
    print(yr)
    data = pd.read_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv')

    filename = './../fitted_RF/clf1_'+yr+'_ARG'
    clf, X_, y_ = fit_save_clf(data, x_cols = x_cols1, y_cols = y_cols1, filename = filename)

    del data
    
    if yr == '2010':
        
        y_pred = y_.copy()
        proba_values = clf.predict_proba(X_)

        ## Leer factores de archivo json
        tag = 'clf1_'+yr+'_ARG'
        with open('./../data/training/factors/'+tag+'.json', 'r') as file:
            info = json.load(file)
        factors_mean = pd.DataFrame(json.loads(info[tag])).stack()
        factors_mean.index.names = ['variable', 'valor']
        factors_mean.index = factors_mean.index.set_levels(factors_mean.index.levels[1].astype(int), level='valor')

        for j, y_col in enumerate(y_cols1):
            y_probas = pd.DataFrame(proba_values[j], columns = sorted(y_[y_col].unique()))
            y_pred[y_col] = (y_probas/factors_mean.loc[y_col]).idxmax(1)#.value_counts().sort_index()

        for col in y_.columns:
            print(col)
            compute_upload_conf_matrix(y_true = y_, y_pred = y_pred, col = col)

2021


In [18]:
print(sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:5])

[('radio_ref', 16764983), ('X_', 11932744), ('y_', 1432056), ('AGLO_rk', 14976), ('Reg_rk', 9948)]


## Classification 2
Columna INGRESO

In [19]:
y_cols2 = predecir2

### Para cada anio. 
- Calcular Factores en Cross Validation y guardar.

In [20]:
for yr in [str(s) for s in range(startyr, endyr)]:
    # Load EPH
    print(yr)
    data = pd.read_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv')
    print(data.shape)

    factors_mean = compute_factors_wCV(data, x_cols = x_cols2, y_cols = y_cols2)
    del data

    ## Guardar factores en archivo json
    tag = 'clf2_'+yr+'_ARG'
    factors_dict = dict()
    factors_dict[tag] = factors_mean.unstack().to_json()

    with open('./../data/training/factors/'+tag+'.json', 'w') as file: 
        json.dump(factors_dict, file)


2021
(357978, 56)
1
2


### Para cada anio. 
- Ajustar modelo y guardar

In [21]:
# Entrenar modelo en train
# for yr in [str(s) for s in range(startyr, endyr)]:

for yr in [str(s) for s in range(2018, endyr)]:
#     # Load EPH
    print(yr)
    data = pd.read_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv')

    filename = './../fitted_RF/clf2_'+yr+'_ARG'
    clf, X_, y_ = fit_save_clf(data, x_cols = x_cols2, y_cols = y_cols2, filename = filename)
    
    del data
    if yr == '2010':
        
        y_pred = y_.copy()
        proba_values = clf.predict_proba(X_)

        ## Leer factores de archivo json
        tag = 'clf2_'+yr+'_ARG'
        with open('./../data/training/factors/'+tag+'.json', 'r') as file:
            info = json.load(file)
        factors_mean = pd.DataFrame(json.loads(info[tag])).stack()
        factors_mean.index.names = ['variable', 'valor']
        factors_mean.index = factors_mean.index.set_levels(factors_mean.index.levels[1].astype(int), level='valor')

        for j, y_col in enumerate(y_cols2):
            y_probas = pd.DataFrame(proba_values[j], columns = sorted(y_[y_col].unique()))
            y_pred[y_col] = (y_probas/factors_mean.loc[y_col]).idxmax(1)#.value_counts().sort_index()

        for col in y_.columns:
            print(col)
            compute_upload_conf_matrix(y_true = y_, y_pred = y_pred, col = col)

2018
2019
2020
2021


## Classification 3
Columnas ['PP07G1', 'PP07G2', 'PP07G3', 'PP07G4', 'PP07G_59', 'PP07H', 'PP07I', 'PP07J', 'PP07K']

In [22]:
y_cols3 = predecir3

### Para cada anio. 
- Calcular Factores en Cross Validation y guardar.

In [23]:
# for yr in [str(s) for s in range(startyr, endyr)]:
for yr in [str(s) for s in range(2018, endyr)]:
    # Load EPH
    print(yr)
    data = pd.read_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv')
    print(data.shape)

    factors_mean = compute_factors_wCV(data, x_cols = x_cols3, y_cols = y_cols3)
    del data

    ## Guardar factores en archivo json
    tag = 'clf3_'+yr+'_ARG'
    factors_dict = dict()
    factors_dict[tag] = factors_mean.unstack().to_json()

    with open('./../data/training/factors/'+tag+'.json', 'w') as file: 
        json.dump(factors_dict, file)


2018
(431568, 56)
1
2
2019
(444320, 56)
1
2
2020
(330618, 56)
1
2
2021
(357978, 56)
1
2


### Para cada anio. 
- Ajustar modelo y guardar

In [24]:
# Entrenar modelo en train
for yr in [str(s) for s in range(startyr, endyr)]:
#     # Load EPH
    print(yr)
    data = pd.read_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv')

    filename = './../fitted_RF/clf3_'+yr+'_ARG'
    clf, X_, y_ = fit_save_clf(data, x_cols = x_cols3, y_cols = y_cols3, filename = filename)
    del data
    
    if yr == '2010':
        
        y_pred = y_.copy()
        proba_values = clf.predict_proba(X_)

        ## Leer factores de archivo json
        tag = 'clf3_'+yr+'_ARG'
        with open('./../data/training/factors/'+tag+'.json', 'r') as file:
            info = json.load(file)
        factors_mean = pd.DataFrame(json.loads(info[tag])).stack()
        factors_mean.index.names = ['variable', 'valor']
        factors_mean.index = factors_mean.index.set_levels(factors_mean.index.levels[1].astype(int), level='valor')

        for j, y_col in enumerate(y_cols3):
            y_probas = pd.DataFrame(proba_values[j], columns = sorted(y_[y_col].unique()))
            y_pred[y_col] = (y_probas/factors_mean.loc[y_col]).idxmax(1)#.value_counts().sort_index()

        for col in y_.columns:
            print(col)
            compute_upload_conf_matrix(y_true = y_, y_pred = y_pred, col = col)

2021


## Regresion
Combinamos la info de los varios trimestres, deflactada.

In [25]:
# startyr = 2003
# endyr = 2004

In [26]:
for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    train = pd.read_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv')

    train[columnas_pesos] = np.log10(train[columnas_pesos].clip(-.9) + 1)

    for q in train.Q.unique():
        print(q)
        ### STEP 3 (Regression)
        train_q = train.loc[train.Q == q]

        X = train_q[x_cols4]#.sample(frac = 1) #PBA_train_reg
        y = train_q[y_cols4].loc[X.index].fillna(0) #PBA_train_reg

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
        X = X_train; y = y_train

        forest = RandomForestRegressor(n_estimators=1, max_depth = 40, n_jobs = -1)
        clf4 = forest.fit(X.values, y.values)
        
        # save the model to disk
        filename = './../fitted_RF/clf4_'+q+'_ARG'
#     pickle.dump(forest, open(filename+'.sav', 'wb'))
        joblib.dump(forest, filename, compress=3)
    
        del clf4;
        del train_q;
    print(sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)[:5])
    del train; del X # liberar memoria eliminando los dataframes mas pesados

2021
2021-08-15
2021-05-15
2021-11-15
2021-02-15
[('train', 231124902), ('X', 24047024), ('X_train', 24047024), ('radio_ref', 16764983), ('X_', 15273872)]
